In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('GPUs.csv', skiprows=1, sep=';')

In [ ]:
dataset.head(30)

In [ ]:
dataset.describe()

# Promethe

In [ ]:
# Preference funcions

def pref_v_shape(d, q, p):
    if d > p:
        return 1
    elif d <= q:
        return 0
    else:
        return (d - q) / (p - q)

In [ ]:
def comprehensive_preference_index(item1, item2, pref_info):
    val = 0.0
    for i in range(len(pref_info)):
        g = pref_info.iloc[i].name
        w = pref_info.iloc[i]["w"]
        q = pref_info.iloc[i]["q"]
        p = pref_info.iloc[i]["p"]
        a = item1[g]
        b = item2[g]
        d = a - b if pref_info.iloc[i]["cost_or_gain"] == "g" else b - a
        pref = pref_info.iloc[i]["pref_func"](d, q, p)
        val += pref * w
    
    val /= pref_info["w"].sum()

    return val 

In [ ]:
pref_info = pd.DataFrame({
    "g": ["Koszt", "Wydajnosc", "TDP", "Ocena"],
    "q": [100, 15, 10, 5],
    "p": [300, 40, 30, 30],
    "w": [3, 4, 1, 2],
    "cost_or_gain": ["c", "g", "c", "g"],
    "pref_func": [pref_v_shape]*4
})

pref_info = pref_info.set_index("g")

In [ ]:
pref_info

In [ ]:
comp_table = []

for i in range(len(dataset)):
    item1 = dataset.iloc[i]
    row = []
    for j in range(len(dataset)):
        item2 = dataset.iloc[j]
        result = comprehensive_preference_index(item1, item2, pref_info)
        row.append(result)
    comp_table.append(row)

comp_df = pd.DataFrame(comp_table)
comp_df

In [ ]:
flow_plus = comp_df.sum(axis=1)
flow_minus = comp_df.sum(axis=0)
flow = flow_plus - flow_minus
flow_df = pd.DataFrame()
flow_df['Nazwa'] = dataset['Nazwa']
flow_df['flow+'] = flow_plus
flow_df['flow-'] = flow_minus
flow_df['flow'] = flow
flow_df

### Promethe I

In [ ]:
flow_df.sort_values("flow+", ascending=False)

In [ ]:
flow_df.sort_values("flow-", ascending=True)

In [ ]:
rank_table = []

for i in range(len(dataset)):
    i1_f_p = flow_df['flow+'][i]
    i1_f_m = flow_df['flow-'][i]
    row = []
    for j in range(len(dataset)):
        i2_f_p = flow_df['flow+'][j]
        i2_f_m = flow_df['flow-'][j]

        result = '' 
        if (i1_f_p > i2_f_p and i1_f_m < i2_f_m) or (i1_f_p == i2_f_p and i1_f_m < i2_f_m) or (i1_f_p > i2_f_p and i1_f_m == i2_f_m):
            result = 'P' 
        elif i1_f_p == i2_f_p and i1_f_m == i2_f_m:
            result = 'I' 
        elif (i1_f_p > i2_f_p and i1_f_m > i2_f_m) or (i1_f_p < i2_f_p and i1_f_m < i2_f_m):
            result = 'R'
        row.append(result)
    rank_table.append(row)

rank_df = pd.DataFrame(rank_table)
rank_df

In [ ]:
def draw_rank(rank_df, flow_df):
    G = nx.DiGraph()

    for i in flow_df.sort_values("flow", ascending=False).index.tolist():
        G.add_node(i)
        for j in range(len(rank_df)):
            if rank_df[j][i] == 'P' and (rank_df[j][rank_df.index[rank_df.loc[i]=='P']] == 'P').sum() == 0:
                G.add_edge(i, j)

    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=500)
    plt.title("Promethe I ranking")
    plt.show()


draw_rank(rank_df, flow_df)

### Promethe II

In [ ]:
flow_df.sort_values("flow", ascending=False)

# Electre

In [ ]:
pref_info = pd.DataFrame({
    "g": ["Koszt", "Wydajnosc", "TDP", "Ocena"],
    "q": [100, 15, 10, 5],
    "p": [300, 40, 30, 30],
    "v": [700, 50, 50, 50],
    "w": [3, 4, 1, 2],
    "cost_or_gain": ["c", "g", "c", "g"],
})

pref_info = pref_info.set_index("g")

In [ ]:
pref_info['w'].iloc[:]

In [ ]:
def marginal_concordance_index_cost(g_a, g_b, q_a, p_a):
    if g_a - g_b <= q_a:
        return 1
    elif g_a - g_b > p_a:
        return 0
    else:
        return (p_a - (g_a - g_b)) / (p_a - q_a)

def marginal_concordance_index_gain(g_a, g_b, q_a, p_a):
    if g_a - g_b >= -q_a:
        return 1
    elif g_a - g_b < -p_a:
        return 0
    else:
        return (p_a - (g_b - g_a)) / (p_a - q_a)

def marginal_discordance_index_cost(g_a, g_b, p_a, v_a):
    if g_a - g_b >= v_a:
        return 1
    elif g_a - g_b <= p_a:
        return 0
    else:
        return (v_a - (g_a - g_b)) / (v_a - p_a)

def marginal_discordance_index_gain(g_a, g_b, p_a, v_a):
    if g_a - g_b <= -v_a:
        return 1
    elif g_a - g_b >= -p_a:
        return 0
    else:
        return ((g_b - g_a) - p_a) / (v_a - p_a)

In [ ]:
def comprehensive_concordance_i(item1, item2, g, pref_info):
    g_a = item1[g]
    g_b = item2[g]
    q_a = pref_info['q'][g]
    p_a = pref_info['p'][g]
    if pref_info['cost_or_gain'][g] == 'c':
        return marginal_concordance_index_cost(g_a, g_b, q_a, p_a)
    else:
        return marginal_concordance_index_gain(g_a, g_b, q_a, p_a)


In [ ]:
def marginal_discordance_i(item1, item2, g, pref_info):
    g_a = item1[g]
    g_b = item2[g]
    p_a = pref_info['p'][g]
    v_a = pref_info['v'][g]
    if pref_info['cost_or_gain'][g] == 'c':
        return marginal_discordance_index_cost(g_a, g_b, p_a, v_a)
    else:
        return marginal_discordance_index_gain(g_a, g_b, p_a, v_a)

In [ ]:
out_cred_table = []

for i in range(len(dataset)):
    item1 = dataset.iloc[i]
    row = []
    for j in range(len(dataset)):
        item2 = dataset.iloc[j]
        C = 0
        W = 0
        D = []
        for g in pref_info.index:
            w = pref_info['w'][g]
            ci = comprehensive_concordance_i(item1, item2, g, pref_info)
            D.append(marginal_discordance_i(item1, item2, g, pref_info))
            C += w * ci
            W += w
        C /= W
        out_cred = C 
        for di in D:
            if di > C:
                out_cred *= (1-di)/(1-C)
        row.append(out_cred)
    out_cred_table.append(row)

out_cred_df = pd.DataFrame(out_cred_table)
out_cred_df

In [ ]:
def s(x):
    return -0.15 * x + 0.3

In [ ]:
def distillation_impl(df, upper_credibility_threshold, desc):
    if upper_credibility_threshold == 0:
        upper_credibility_threshold = df.max().max()
    s_k = s(upper_credibility_threshold)
    lower_credibility_threshold = df[df < upper_credibility_threshold-s_k].max().max()
    if np.isnan(lower_credibility_threshold):
        lower_credibility_threshold = 0
    relations = (df > lower_credibility_threshold) & (df > (df.transpose() + df.apply(s)))
    survivers = df[relations]
    quality = (survivers > 0).apply(np.sum, axis=1) - (survivers > 0).apply(np.sum, axis=0)
    if desc:
        return quality == quality.max(), lower_credibility_threshold
    else:
        return quality == quality.min(), lower_credibility_threshold

def distillation(df_in, desc=True):
    df = df_in.copy()
    D = []
    while len(df) > 0:
        d1, lower_thr = distillation_impl(df, 0, desc)
        d = d1
        if d1.sum() > 1:
            d2, _ = distillation_impl(df.loc[d1, d1], lower_thr, desc)
            d[d == True] = d2
            d = d.astype(bool)
        D.append(list(df[d].index))
        df = df.loc[~d, ~d]

    if desc:
        return D
    else:
        return list(reversed(D))

In [ ]:
def print_ranking(ranking):
    for i,place in enumerate(ranking):
        print(f'{i+1}. {[" ".join(x.split()) for x in dataset["Nazwa"].iloc[place]]}')

In [ ]:
def get_median_ranking(dataset, ranking_descending, ranking_ascending):
    ranking_values = {}
    for i, items in enumerate(ranking_descending):
        for item in items:
            ranking_values[item] = i

    for i, items in enumerate(ranking_ascending):
        for item in items:
            ranking_values[item] += i

    return [x[0] for x in sorted([(v,k) for v,k in ranking_values.items()], key=lambda x: x[1])]


In [ ]:
ranking_descending = distillation(out_cred_df, True)
print_ranking(ranking_descending)

In [ ]:
ranking_ascending = distillation(out_cred_df, False)
print_ranking(ranking_ascending)

In [ ]:
print_ranking(get_median_ranking(dataset ,ranking_descending, ranking_ascending))